In [7]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.read_excel('../../Produktion/TrainingData/TrainingData_5cat_de.xlsx')
# df2 = pd.read_excel('../webserver/model_data/TrainingData_5cat_de.xlsx')


print("Column headings:")
print(df.columns)

Column headings:
Index(['Unnamed: 0', 'platform', 'phase/arg_situation', 'method',
       'adressee, reference', 'function', 'intent', 'impl_expl', 'pos_neg',
       'easy translation', 'User', 'topic', 'link', 'connected', 'utterance',
       'Effekt', 'lecture', 'praise', 'insinuation', 'dissence', 'concession'],
      dtype='object')


In [8]:
from collections import Counter, OrderedDict

category_names = Counter()
for i in df.index:
    category_names[df['Effekt'][i]] +=1

for entry in category_names.most_common():
    print(entry)


('lecture', 287)
('insinuation', 270)
('dissence', 232)
('concession', 220)
('praise', 215)


In [9]:
import re 

def test_patterns(text, patterns):
    """Given source text and a list of patterns, look for
    matches for each pattern within the text and print
    them to stdout.
    """

    # Look for each pattern in the text and print the results
    for pattern in patterns:
        pattern = re.compile(pattern, re.VERBOSE)
        print('{}'.format(pattern))
        print('{}'.format(text))
        for match in re.finditer(pattern, text):
            s = match.start()
            e = match.end()
            prefix = ' ' * (s)
            print(
                'MATCHED! {}{}{} '.format(prefix,
                                     text[s:e],
                                     ' ' * (len(text) - e)),
                end=' ',
            )
            print(match.groups())
            if match.groupdict():
                print('{}{}'.format(
                    ' ' * (len(text) - s),
                    match.groupdict()),
                )
        print()
    return

patterns = [ 
    '''
    .*           # anything
    \(           # starts with a paranthesis
    \w+          # followed by a word 
    (?=\|)       # positive LOOKAHEAD. word must be followed by pipe
    [\w+\|\w+]+  # A character set
    \).*           # ends with paranthesis
    ''', 
    r'\.\*',
    '''Das\ssind\s(keine|zwei|eine)\sEnteignungen,\snur\sdie\s(Gerechtigkeit|Vollzug|Urinente|Grundrente),
    \sdas\s(ist|wäre)\sVolksherrschaft 
    ''',
    '''.* Das sind (keine|zwei|eine) Enteignungen, 
    .* nur die (Gerechtigkeit|Vollzug|Urinente|Grundrente), 
    das (ist|wäre) Volksherrschaft (Demokratie)''',
    r'.*\(\w+.*\)'
    
]
text = "ja, Das sind keine Enteignungen, nur die Urinente, das wäre Volksherrschaft"
pattern3 = patterns[3]
pattern3 = re.sub(r'\s+', r'\\s', pattern3)
pattern3 = re.sub(r'\.\*', r'\.\*', pattern3)
print('pattern3: ',pattern3)
patterns[3] = pattern3

test_patterns(text, patterns)


pattern3:  \.\*\sDas\ssind\s(keine|zwei|eine)\sEnteignungen,\s\.\*\snur\sdie\s(Gerechtigkeit|Vollzug|Urinente|Grundrente),\sdas\s(ist|wäre)\sVolksherrschaft\s(Demokratie)
re.compile('\n    .*           # anything\n    \\(           # starts with a paranthesis\n    \\w+          # followed by a word \n    (?=\\|)       # positive LOOKAHEAD. word must be followed by pipe\n    [\\w+, re.VERBOSE)
ja, Das sind keine Enteignungen, nur die Urinente, das wäre Volksherrschaft

re.compile('\\.\\*', re.VERBOSE)
ja, Das sind keine Enteignungen, nur die Urinente, das wäre Volksherrschaft

re.compile('Das\\ssind\\s(keine|zwei|eine)\\sEnteignungen,\\snur\\sdie\\s(Gerechtigkeit|Vollzug|Urinente|Grundrente),\n    \\sdas\\s(ist|wäre)\\sVolksherrschaft \n    ', re.VERBOSE)
ja, Das sind keine Enteignungen, nur die Urinente, das wäre Volksherrschaft
MATCHED!     Das sind keine Enteignungen, nur die Urinente, das wäre Volksherrschaft  ('keine', 'Urinente', 'wäre')

re.compile('\\.\\*\\sDas\\ssind\\s(keine|z

In [10]:
text = ".* Das sind (keine|zwei|eine) Enteignungen, .* nur die (Gerechtigkeit|Vollzug|Urinente|Grundrente), das (ist|wäre) Volksherrschaft (Demokratie)"
text2 = "Das sind keine Enteignungen, nur die Gerechtigkeit, das ist Volksherrschaft (Demokratie)"
patterns = [ 
    '''
    \(           # starts with a paranthesis
    \w+
    (?=\|)      # positive LOOKAHEAD. word must  be followed by pipe
    [\w+\|\w+]+  # A character set
    \)           # ends with paranthesis
    ''', 
    '''
    \(            # starts with a paranthesis
    \w+           # A word
    (?!\))         # negative LOOKAHEAD. word must not be followed by parathesis 
    [\w+\|\w+]+
    \)            # ends with paranthesis
    ''',
    '''
    \(                # starts with a paranthesis
    (\w+)             # followed by word 
    (?=\|)         # positive LOOKAHEAD: word only followed by pipe followed by word 
    (\|\w+)+          # followed by (pipe followed by word) one or more
    \)               # ends with paranthesis
    ''',
     '''
    \(                 # starts with a paranthesis
    ((\w)+
    (?!(\w+[^\|]$))   # negative LOOKAHEAD: word not followed by pipe
    )
    \)                 # ends with paranthesis
    ''',
     '''
    \(              # starts with a paranthesis
    [(?<=\|)\w+]+   # alphanumeric chars followed by a "|"
    \)              # ends with paranthesis
    ''',
    r'\((\w+)\)',   
    r'\.\*', 
    
]

for pattern in patterns:
    regex = re.compile(pattern, re.VERBOSE)
    m = re.search(pattern, text)
    # print('findall: ', regex.findall(text))
    # print('match: ',m)

regex = re.compile(patterns[0], re.VERBOSE)
match = re.search(regex, text2)
gava = df[['utterance', 'topic', 'Effekt']]
gava.to_csv('gavagai_test.csv', sep=';')


for utt, topic, sent in zip(gava['utterance'], gava['topic'], gava['Effekt']):

    regex = re.compile(patterns[0], re.VERBOSE)
    if regex.search(utt):
        print('regex: {}\ntopic: {}\nsentiment: {}'.format(utt, topic, sent))
    print('verbatim: {}\ntopic: {} \nsentiment: {} '.format(utt, topic, sent))


verbatim: Merkel war und ist hier eine getriebene
topic: human_rights 
sentiment: insinuation 
verbatim: Das ist doch albern.
topic: unknown 
sentiment: lecture 
verbatim: Dass ich nicht lache
topic: unknown 
sentiment: dissence 
verbatim: Sie haben schlicht gar nichts begriffen.
topic: unknown 
sentiment: lecture 
verbatim: Stellen Sie sich nicht dumm
topic: unknown 
sentiment: lecture 
verbatim: Paragraphen-Nachhilfe für Nazis
topic: politics 
sentiment: insinuation 
verbatim: DON'T FEED THE TROLL!!
topic: nature 
sentiment: insinuation 
verbatim: Dann werden Sie sicherlich verstehen, warum von diesen keine neutrale Untersuchung der Hamas-Attacken zu erwarten ist.
topic: nan 
sentiment: lecture 
verbatim: Sie sollten sich mein Posting noch einmal in Ruhe anschauen
topic: nan 
sentiment: lecture 
verbatim: Gott sei dank, Dankeschön!
topic: nan 
sentiment: praise 
verbatim: ich entfreunde mich jetzt , so einen infantilen Scheiß kann ich nicht mehr ertragen
topic: public_space 
sentimen

sentiment: lecture
verbatim: .* (du|sie) .* (solltest|sollten) .* zur Kenntnis genommen haben .*
topic: any 
sentiment: lecture 
regex: .* (du|sie) .* (dürften|dürftest|solltest|sollten) .* zur Kenntnis genommen haben .*
topic: any
sentiment: insinuation
verbatim: .* (du|sie) .* (dürften|dürftest|solltest|sollten) .* zur Kenntnis genommen haben .*
topic: any 
sentiment: insinuation 
regex: .* (du|sie) .* (solltest|sollten) sich (klar|bewusst) darüber .* (werden|sein) .*
topic: any
sentiment: lecture
verbatim: .* (du|sie) .* (solltest|sollten) sich (klar|bewusst) darüber .* (werden|sein) .*
topic: any 
sentiment: lecture 
verbatim: Vielleicht sollten sie auf Kommentare weniger bewertend eingehen, sondern sich mehr inhaltlich damit auseinander setzen
topic: any 
sentiment: lecture 
regex: .* Vielleicht .* (solltest|sollten) .* (du|sie)  .* sich .* inhaltlich damit auseinander setzen
topic: any
sentiment: lecture
verbatim: .* Vielleicht .* (solltest|sollten) .* (du|sie)  .* sich .* inhalt

sentiment: concession 
verbatim: Ein guter Beleg dafür, dass es bei vielen Leuten (vor allem bei Rechtsauslegern) nicht weit her ist mit Sarkasmusgespür und Humorerkennung.
topic: nan 
sentiment: insinuation 
verbatim: Da sind wir also.
topic: nan 
sentiment: lecture 
verbatim: Neinnein, die spielt damit.
topic: nan 
sentiment: insinuation 
verbatim: Darum geht es aber im Artikel nicht.
topic: nan 
sentiment: dissence 
verbatim: Da geht es um die Einsichtsfähigkeit von Tätern.
topic: nan 
sentiment: lecture 
verbatim: dann ist das keineswegs eine Anklage gegen Deutschland.
topic: nan 
sentiment: dissence 
verbatim: Das ist keine Anklage gegen Deutschland, sondern eine Anklage gegen die Regime im Nahen Osten.
topic: nan 
sentiment: dissence 
verbatim: Das ist der Punkt. 
topic: nan 
sentiment: lecture 
verbatim: Wo inhaltlich nichts entgegengesetzt werden kann, wird immer an der Form rumgemeckert.
topic: nan 
sentiment: insinuation 
verbatim: Das kommt immer nur, wenn die Meinung nicht 

sentiment: praise 
verbatim: Top
topic: art 
sentiment: praise 
verbatim: Sehr cool!
topic: art 
sentiment: praise 
verbatim: Es lohnt sich
topic: art 
sentiment: praise 
verbatim: Eine kluge Analyse! Vielen Dank!
topic: politics 
sentiment: praise 
verbatim: Volle Zustimmung. Danke für diesen Artikel.
topic: politics 
sentiment: praise 
verbatim: Der Artikel bringt es auf den Punkt
topic: politics 
sentiment: praise 
verbatim: Man muss Höcke nur sehr genau zuhören, dann offenbart sich schnell, dass sich unter einer dünnen demokratischen Firnis ein zutiefst rassistisches Weltbild verbirgt
topic: politics 
sentiment: concession 
verbatim: Schon. In diesem Fall glaube ich allerdings eher, dass der Begriff "Faschist" vielleicht sogar kontraproduktiv ist
topic: politics 
sentiment: concession 
verbatim: Größten Respekt für die hervorragend umfangreiche und tiefgehende Recherche
topic: media 
sentiment: praise 
verbatim: Toller Beitrag!
topic: media 
sentiment: praise 
verbatim: Sehr intere